## -- Notebook: 01_delta_table_optimizations.ipynb
-- Objetivo: Explorar técnicas de otimização para tabelas Delta.

## -- Vamos usar a tabela Gold (mc_labs.daily_city_pair_metrics) e Silver (mc_labs.trips_silver)
-- como exemplos para aplicar as otimizações.


In [0]:
%sql

-- 1. Entendendo o estado atual da tabela (antes da otimização)
-- Podemos ver quantos arquivos a tabela tem e seu tamanho
DESCRIBE DETAIL mc_labs.daily_city_pair_metrics;


## -- Ou, para uma visão mais detalhada dos arquivos (requer permissões de acesso ao sistema de arquivos)
-- SELECT * FROM json.`${path_to_delta_table}/_delta_log/00000000000000000000.json` -- Exemplo de como ver o log
-- No Databricks, o DESCRIBE DETAIL já dá uma boa ideia.

## -- 2. OPTIMIZE: Compactação de Arquivos Pequenos
-- O comando OPTIMIZE compacta arquivos de dados pequenos em arquivos maiores,
-- o que reduz a sobrecarga de leitura e melhora o desempenho das consultas.
-- É especialmente útil após muitas operações de INSERT, UPDATE ou MERGE INTO.


In [0]:
%sql
-- Sintaxe básica: OPTIMIZE table_name;
-- Sintaxe com filtro (para otimizar apenas partes da tabela): OPTIMIZE table_name WHERE condition;

-- Exemplo: Otimizar a tabela Gold
OPTIMIZE mc_labs.daily_city_pair_metrics;

In [0]:
%sql
-- Verifique novamente o DESCRIBE DETAIL para ver a mudança no número de arquivos
DESCRIBE DETAIL mc_labs.daily_city_pair_metrics;
-- Você deve notar uma redução no número de arquivos e um aumento no tamanho médio dos arquivos.

## -- 3. Z-Ordering: Otimização para Consultas com Filtros
-- Z-Ordering é uma técnica de co-localização de dados que organiza dados relacionados
-- em um conjunto de arquivos. Isso é extremamente eficaz para consultas que filtram
-- por uma ou mais colunas, pois o Spark pode pular arquivos inteiros que não contêm
-- os dados relevantes.

-- É uma extensão do comando OPTIMIZE.
-- Escolha colunas que são frequentemente usadas em cláusulas WHERE ou JOIN.
-- Não use muitas colunas (geralmente 1 a 4 é o ideal).

In [0]:
%sql
-- Exemplo: Aplicar Z-Ordering na tabela Gold por pickup_date e pickup_city_name
-- Estas são colunas que provavelmente serão usadas para filtrar relatórios de BI.
OPTIMIZE mc_labs.daily_city_pair_metrics
ZORDER BY (pickup_date, pickup_city_name);

In [0]:
%sql
-- Exemplo: Aplicar Z-Ordering na tabela Gold por pickup_date e pickup_city_name
-- Estas são colunas que provavelmente serão usadas para filtrar relatórios de BI.
OPTIMIZE mc_labs.daily_city_pair_metrics
ZORDER BY (pickup_date, pickup_city_name);

## -- 4. Particionamento (Revisão e Considerações com Delta Lake)
-- O particionamento organiza os dados em subdiretórios com base nos valores de uma ou mais colunas.
-- No Delta Lake, o particionamento ainda é suportado, mas o Z-Ordering é frequentemente preferível
-- para muitas cargas de trabalho, especialmente com um grande número de partições.

## -- Quando usar particionamento:
-- - Quando você tem um número relativamente pequeno de valores distintos para a coluna de partição (ex: ano, mês).
-- - Quando suas consultas filtram *quase sempre* por essa coluna e você quer que o Spark ignore grandes volumes de dados.
-- - Quando você precisa gerenciar o ciclo de vida dos dados em nível de partição (ex: arquivar dados antigos por ano).


## -- Como criar uma tabela Delta particionada (exemplo, não execute se já criou a tabela sem partição):


In [0]:
%sql
CREATE TABLE mc_labs.daily_city_pair_metrics_partitioned
USING DELTA
PARTITIONED BY (pickup_date) -- Ou (pickup_year, pickup_month)
AS SELECT
    pickup_date,
    pickup_city_name,
    dropoff_city_name,
    COUNT(*) AS total_trips,
    AVG(fare_amount) AS avg_fare_amount,
    SUM(trip_distance) AS total_trip_distance
FROM
    mc_labs.trips_silver
GROUP BY
    pickup_date,
    pickup_city_name,
    dropoff_city_name;

## -- Para tabelas existentes, você não pode simplesmente adicionar PARTITIONED BY.
-- Você precisaria recriar a tabela ou usar um comando como `ALTER TABLE ... ADD PARTITION` (que é mais para Hive).
-- No Delta Lake, a melhor prática é definir o particionamento na criação da tabela.

-- **Diferença chave entre Z-Ordering e Particionamento:**
-- - Particionamento: Cria diretórios físicos. Ótimo para filtros de alta seletividade com poucos valores distintos. Pode levar a muitos arquivos pequenos se houver muitas partições.
-- - Z-Ordering: Organiza dados *dentro* dos arquivos. Ótimo para filtros de média/baixa seletividade e para evitar o problema de "muitos arquivos pequenos" que o particionamento pode causar.


## -- 5. VACUUM: Limpeza de Arquivos Antigos
-- O comando VACUUM remove arquivos de dados que não são mais referenciados pelo log de transações
-- do Delta Lake e que estão além de um determinado limite de retenção.
-- Isso libera espaço de armazenamento.


## -- **ATENÇÃO:** Use VACUUM com cautela!
-- - Ele remove arquivos permanentemente.
-- - Se você usar VACUUM, o Time Travel só funcionará para versões dentro do período de retenção.
-- - O padrão de retenção é 7 dias. Você pode alterá-lo com `SET spark.databricks.delta.retentionDurationCheck.enabled = false;` e `VACUUM table_name RETAIN 0 HOURS;` (mas isso é perigoso).

-- Exemplo: Remover arquivos não referenciados com um período de retenção de 0 horas (apenas para demonstração, não recomendado em produção sem entender o impacto)
-- SET spark.databricks.delta.retentionDurationCheck.enabled = false; -- Desabilita a verificação de retenção para permitir 0 horas
-- VACUUM mc_labs.daily_city_pair_metrics RETAIN 0 HOURS;
-- SET spark.databricks.delta.retentionDurationCheck.enabled = true; -- Reabilita a verificação

-- Para uso em produção, geralmente você executa VACUUM com o período de retenção padrão (7 dias)
-- ou um período maior que garanta que o Time Travel ainda funcione para suas necessidades.
-- VACUUM mc_labs.daily_city_pair_metrics; -- Usará o padrão de 7 dias


In [0]:
%sql
-- Para ver os arquivos que seriam removidos sem realmente removê-los:
VACUUM mc_labs.daily_city_pair_metrics DRY RUN;


## -- 6. Analisar e Otimizar Consultas
-- Além das otimizações de tabela, é crucial analisar o plano de execução das suas consultas.
-- Use EXPLAIN para entender como o Spark está processando sua query.

In [0]:
%sql
EXPLAIN SELECT * FROM mc_labs.daily_city_pair_metrics WHERE pickup_city_name = 'New York' AND pickup_date = '2023-01-01';

-- Observe no plano de execução se o Spark está fazendo "file skipping" (pulando arquivos)
-- graças ao Z-Ordering e aos metadados do Delta Lake.